In [1]:
import time
# import pandas as pd
import pandas as pd

from save_keypoints import get_options, KeyPointExtractors
import argparse

import open3d as o3d
import numpy as np
import copy

import os

import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def getUnixTime():
    return int(time.time())

class PoseGraphResultSaver:
    def __init__(self, init_pose, save_gap, num_frames, seq_idx, save_dir):
        self.pose_list = np.reshape(init_pose, (-1, 16))
        self.gt_pose_list = np.reshape(init_pose, (-1, 16))
        self.save_gap = save_gap
        self.num_frames = num_frames

        self.seq_idx = seq_idx
        self.save_dir = save_dir
        
        self.errors = []

    def saveUnoptimizedPoseGraphResult(self, cur_pose, curr_gt_pose, cur_node_idx):
        # save 
        self.pose_list = np.vstack((self.pose_list, np.reshape(cur_pose, (-1, 16))))
        
        self.gt_pose_list= np.vstack((self.gt_pose_list, np.reshape(curr_gt_pose, (-1, 16))))

    def vizCurrentTrajectory(self, fig_idx, title, legends={}):
        x = self.pose_list[:,3]
        y = self.pose_list[:,7]
        z = self.pose_list[:,11]

        print("pred" , x[-1],y[-1],z[-1])
        if "nosample" in title:
            curr_pred = np.array([-self.pose_list[-1][7], -self.pose_list[-1][11], self.pose_list[-1][3]])
        else:
            curr_pred = np.array( [self.pose_list[-1][3], self.pose_list[-1][7], self.pose_list[-1][11]])
        curr_pred = np.array( [self.pose_list[-1][3], self.pose_list[-1][7], self.pose_list[-1][11]])

                        
        curr_gt = np.array( [self.gt_pose_list[-1][3], self.gt_pose_list[-1][7], self.gt_pose_list[-1][11]])
        ate = self.compute_ATE(curr_gt, curr_pred)
        # Create empty plot with blank marker containing the extra label


        fig = plt.figure(fig_idx)
        
        plt.clf()
        if "kitti" in title and "nosample" in title:
            plt.plot(-y, x, color='blue')
        elif "synth" in title:
            plt.plot(x,y, color='blue')
        else:
            plt.plot(x,z, color='blue')


        #plt.plot(x,z, color='blue') # kitti camera coord for clarity
        #-y, x
        plt.axis('equal')
        plt.xlabel('x', labelpad=10)
        plt.ylabel('y', labelpad=10)
        
        #print(title)
        #print(self.gt_pose_list[-1])
        
        if 'kitti' in title or 'oxford' in title or 'luco' in title:
            x = self.gt_pose_list[:,3]
            y = self.gt_pose_list[:,7]
            z = self.gt_pose_list[:,11] 
            plt.plot(x,z, color='green')    
            #plt.plot(-y,-x, color='green')
            print("gt" ,x[-1],y[-1],z[-1])
        else:
            x = self.gt_pose_list[:,3]
            y = self.gt_pose_list[:,7]
            z = self.gt_pose_list[:,11] 
            plt.plot(x,y, color='green')    
            #plt.plot(-y,-x, color='green')
            print("gt" ,x[-1],y[-1],z[-1])            
        
        legends ["ATE"] = ate
        
        for legnd in legends:
            plt.plot([], [], ' ', label=legnd +" : "+str(float(f'{legends[legnd]:.4f}')))
        
        plt.legend()
        plt.title(title)
        
        plt.draw()
        plt.pause(0.01) #is necessary for the plot to update for some reason

        return ate

    def compute_ATE(self, cur_gt, cur_pred):


        gt_xyz = cur_gt #[:3, 3] 

        pred_xyz = cur_pred #[:3, 3]

        align_err = gt_xyz - pred_xyz

        # print('i: ', i)
        # print("gt: ", gt_xyz)
        # print("pred: ", pred_xyz)
        # input("debug")
        self.errors.append(np.sqrt(np.sum(align_err ** 2)))
        
        ate = np.sqrt(np.mean(np.asarray(self.errors) ** 2)) 
        return ate

In [3]:


# params
parser = argparse.ArgumentParser(description='PyICP SLAM arguments')

parser.add_argument('--num_icp_points', type=int, default=5000) # 5000 is enough for real time

parser.add_argument('--num_rings', type=int, default=20) # same as the original paper
parser.add_argument('--num_sectors', type=int, default=60) # same as the original paper
parser.add_argument('--num_candidates', type=int, default=10) # must be int
parser.add_argument('--try_gap_loop_detection', type=int, default=10) # same as the original paper

parser.add_argument('--loop_threshold', type=float, default=0.11) # 0.11 is usually safe (for avoiding false loop closure)

parser.add_argument('--data_base_dir', type=str, 
                    default='/home/pranayspeed/Work/git_repos/data_odometry_velodyne_part/sequences')

parser.add_argument('--ground_truth_dir', type=str, 
                    default='/home/pranayspeed/Work/git_repos/data_odometry_velodyne_part/poses')
parser.add_argument('--sequence_idx', type=str, default='00')

parser.add_argument('--save_gap', type=int, default=300)

parser.add_argument('--mesh_path', type=str, 
                    default='mesh.ply')

parser.add_argument('--mesh_map', action='store_true')

parser.add_argument('--show_map', action='store_true')


parser.add_argument('--method',  type=str, 
                    default="usip")

parser.add_argument('--sequence',  type=int, 
                    default=0)

# parser.add_argument('--dataset',  type=str, 
#                     default="kitti")


_StoreAction(option_strings=['--sequence'], dest='sequence', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, help=None, metavar=None)

In [4]:
opt_detector = get_options(parser)
args = opt_detector# .opt# parser.parse_args()

not none arg_parser
------------ Options -------------
activation: relu
batch_size: 1
bn_momentum: 0.1
bn_momentum_decay: 0.6
bn_momentum_decay_step: None
checkpoints_dir: ./checkpoints
data_base_dir: /home/pranayspeed/Work/git_repos/data_odometry_velodyne_part/sequences
dataroot: /ssd/jiaxin/USIP_datasets/kitti
dataset: kitti
device: cuda:0
display_id: 1
display_winsize: 256
gpu_ids: [0]
ground_truth_dir: /home/pranayspeed/Work/git_repos/data_odometry_velodyne_part/poses
input_pc_num: 16384
k: 1
keypoint_on_pc_alpha: 0.01
keypoint_on_pc_type: point_to_point
keypoint_outlier_thre: 3
loop_threshold: 0.11
loss_sigma_lower_bound: 0.001
lr: 0.001
mesh_map: False
mesh_path: mesh.ply
method: usip
nThreads: 1
name: train
node_knn_k_1: 16
node_num: 512
normalization: batch
num_candidates: 10
num_icp_points: 5000
num_rings: 20
num_sectors: 60
radius_threshold: 100
random_pc_dropout_lower_limit: 1
rot_3d: False
rot_horizontal: True
rot_perturbation: False
save_gap: 300
scene: outdoor
sequence: 0

In [5]:
curr_transform = np.identity(4)

num_frames = 600
num_frames_to_skip_to_show = 1
# Result saver
save_dir = "result/" + args.sequence_idx
if not os.path.exists(save_dir): os.makedirs(save_dir)
ResultSaver = PoseGraphResultSaver(init_pose=curr_transform, 
                             save_gap=args.save_gap,
                             num_frames=num_frames,
                             seq_idx=args.sequence_idx,
                             save_dir=save_dir)

In [6]:



keypoint_extractor_name =  args.method
    
seq_name = args.dataset+"_"+args.sequence_idx + "_" + keypoint_extractor_name
#if args.method =='ramdom' or args.method =='uniform' :
seq_name+= "_" + str(args.num_icp_points)
###########################################
for_idx=-1
frame_limit=600

In [7]:
from scipy.spatial.transform import Rotation as R

import random

def get_random_transform():
    point_cloud = o3d.geometry.PointCloud()
    euler_angles = R.random().as_euler('zxy', degrees=False) #random radian rotation


    list_Size = 3
    # random float from 1 to 99.9
    integer_list = random.sample(range(1, 20), list_Size)
    float_list = [x/10 for x in integer_list]
              
    T = np.eye(4)
    T[:3, :3] = point_cloud.get_rotation_matrix_from_xyz(euler_angles)
    T[0, 3] = float_list[0]
    T[1, 3] = float_list[1]
    T[2, 3] = float_list[2]
    return T

def get_transformed_point_cloud(point_cloud, transform):
    # point_cloud = o3d.geometry.PointCloud()
    # point_cloud.points = o3d.utility.Vector3dVector(curr_pts) 

    point_cloud_transformed = copy.deepcopy(point_cloud).transform(transform)
    
    return point_cloud , point_cloud_transformed
    

def extract_keypoints_ISS(input_pcd, anc_pc, anc_sn, anc_node, farthest_sampler) :
    tic = time.time()
    keypoints = o3d.geometry.keypoint.compute_iss_keypoints(input_pcd)
    toc = 1000 * (time.time() - tic)
    #print("ISS Computation took {:.0f} [ms]".format(toc))     
    
    return keypoints, toc 


import PCLKeypoint

def extract_keypoints_SIFT(input_pcd, anc_pc, anc_sn, anc_node, farthest_sampler) :
    tic = time.time()
    # method = 'sift'
    min_scale = 0.5
    n_octaves = 4
    n_scales_per_octave = 8
    min_contrast = 0.1    
    keypoints = PCLKeypoint.keypointSift(input_pcd.points,
                                                             min_scale,
                                                             n_octaves,
                                                             n_scales_per_octave,
                                                             min_contrast)  # Mx3
    
    toc = 1000 * (time.time() - tic)
    #print("SIFT Computation took {:.0f} [ms]".format(toc), " count :", len(keypoints))     
    point_cloud_kps = o3d.geometry.PointCloud()
    point_cloud_kps.points = o3d.utility.Vector3dVector(keypoints) 
    
        
    return point_cloud_kps, toc     


def extract_keypoints_Harris(input_pcd, anc_pc, anc_sn, anc_node, farthest_sampler) :
    tic = time.time()
    # method = 'harris'
    radius = 1
    nms_threshold = 0.001
    threads = 0   
    keypoints = PCLKeypoint.keypointHarris3D(input_pcd.points,
                                                               radius,
                                                               nms_threshold,
                                                               threads,
                                                               True)  # Mx3
    
    toc = 1000 * (time.time() - tic)
    #print("Harris Computation took {:.0f} [ms]".format(toc), " count :", len(keypoints))     
    point_cloud_kps = o3d.geometry.PointCloud()
    point_cloud_kps.points = o3d.utility.Vector3dVector(keypoints) 
    
        
    return point_cloud_kps, toc 
    
def compute_inverse_transform_pcd(input_point_cloud, transfrm):
    inv_transfm = np.linalg.inv(transfrm)
    point_cloud_transformed = copy.deepcopy(input_point_cloud).transform(inv_transfm)
    return point_cloud_transformed, inv_transfm


from sklearn.neighbors import KDTree
from sklearn.neighbors import NearestNeighbors

def compute_repeatablity(pc1, pc2, search_rad=0.5):
    matches = get_matches_pcd(pc1, pc2, search_rad)
    matches = np.unique(matches)
    if len(pc1.points) ==0:
        return 1.0
    #print("Matches : ", len(matches), "/", len(pc1.points))
    repeatiblity_res = len(matches)/ len(pc1.points)
    #print("Repeatabilty: ", repeatiblity_res)
    return repeatiblity_res, len(pc1.points), len(matches)

def get_matches_pcd(pc1, pc2, search_rad=0.5):
    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(pc2.points)
    distances, indices = neigh.kneighbors(pc1.points, return_distance=True)
    matches = indices.flatten()  
    distances = distances.flatten()
    final_matches = [matches[i] for i in range(len(matches)) if distances[i]<search_rad]  
    return final_matches  

def print_matches_pcd(pc1, pc2, search_rad=0.0001):

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(pc2.points)
    distances, indices = neigh.kneighbors(pc1.points, return_distance=True)
    matches = indices.flatten()  
    distances = distances.flatten()
    final_matches = [matches[i] for i in range(len(matches)) if distances[i]<search_rad]  
    print(final_matches)
    
    
    
    


In [8]:
from models.keypoint_detector import ModelDetector
import torch

from evaluation.kitti_test_loader import FarthestSampler

def model_state_dict_parallel_convert(state_dict, mode):
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    if mode == 'to_single':
        for k, v in state_dict.items():
            name = k[7:]  # remove 'module.' of DataParallel
            new_state_dict[name] = v
    elif mode == 'to_parallel':
        for k, v in state_dict.items():
            name = 'module.' + k  # add 'module.' of DataParallel
            new_state_dict[name] = v
    elif mode == 'same':
        new_state_dict = state_dict
    else:
        raise Exception('mode = to_single / to_parallel')

    return new_state_dict


def model_state_dict_convert_auto(state_dict, gpu_ids):
    for k, v in state_dict.items():
        if (k[0:7] == 'module.' and len(gpu_ids) >= 2) or (k[0:7] != 'module.' and len(gpu_ids) == 1):
            return state_dict
        elif k[0:7] == 'module.' and len(gpu_ids) == 1:
            return model_state_dict_parallel_convert(state_dict, mode='to_single')
        elif k[0:7] != 'module.' and len(gpu_ids) >= 2:
            return model_state_dict_parallel_convert(state_dict, mode='to_parallel')
        else:
            raise Exception('Error in model_state_dict_convert_auto')


def nms(keypoints_np, sigmas_np, NMS_radius):
    '''

    :param keypoints_np: Mx3
    :param sigmas_np: M
    :return: valid_keypoints_np, valid_sigmas_np, valid_descriptors_np
    '''
    if NMS_radius < 0.01:
        return keypoints_np, sigmas_np

    valid_keypoint_counter = 0
    valid_keypoints_np = np.zeros(keypoints_np.shape, dtype=keypoints_np.dtype)
    valid_sigmas_np = np.zeros(sigmas_np.shape, dtype=sigmas_np.dtype)

    while keypoints_np.shape[0] > 0:
        # print(sigmas_np.shape)
        # print(sigmas_np)

        min_idx = np.argmin(sigmas_np, axis=0)
        # print(min_idx)

        valid_keypoints_np[valid_keypoint_counter, :] = keypoints_np[min_idx, :]
        valid_sigmas_np[valid_keypoint_counter] = sigmas_np[min_idx]
        # remove the rows that within a certain radius of the selected minimum
        distance_array = np.linalg.norm(
            (valid_keypoints_np[valid_keypoint_counter:valid_keypoint_counter + 1, :] - keypoints_np), axis=1,
            keepdims=False)  # M
        mask = distance_array > NMS_radius  # M

        keypoints_np = keypoints_np[mask, ...]
        sigmas_np = sigmas_np[mask]

        # increase counter
        valid_keypoint_counter += 1

    return valid_keypoints_np[0:valid_keypoint_counter, :], \
           valid_sigmas_np[0:valid_keypoint_counter]

def extract_keypoints_usip(input_pcd, anc_pc, anc_sn, anc_node, farthest_sampler) :

    detector_model_path = '/home/pranayspeed/Work/git_repos/TSF_datasets/oxford-checkpoints/oxford/16384-256-k1k32-2d/best.pth'
    # build detector
    model_detector = ModelDetector(opt_detector)
    model_detector.detector.load_state_dict(
        model_state_dict_convert_auto(
            torch.load(
                detector_model_path,
                map_location='cpu'), opt_detector.gpu_ids))
    model_detector.freeze_model()      


    pc_np = np.asarray(input_pcd.points)
    normals_np = np.asarray(input_pcd.normals)
    
    anc_sn_new = anc_sn.detach().numpy().transpose()
    
    sn_np = np.append(normals_np, anc_sn_new[:,3],1)

    desired_keypoint_num = 128
    NMS_radius = 0
    noise_sigma = 0
    downsample_rate = 1    

    
    # get nodes, perform random sampling to reduce computation cost
    node_np = farthest_sampler.sample(
        pc_np[np.random.choice(pc_np.shape[0], int(opt_detector.input_pc_num / 4), replace=False)],
        opt_detector.node_num)


    # convert to torch tensor
    anc_pc1 = torch.from_numpy(pc_np.transpose().astype(np.float32))  # 3xN
    anc_sn1 = torch.from_numpy(sn_np.transpose().astype(np.float32))  # 4xN
    anc_node1 = torch.from_numpy(node_np.transpose().astype(np.float32))  # 3xM


    # anc_pc_cuda = anc_pc.to(opt_detector.device)
    # anc_sn_cuda = anc_sn.to(opt_detector.device)
    # anc_node_cuda = anc_node.to(opt_detector.device)

    anc_pc_cuda = torch.unsqueeze(anc_pc1, dim=0).to(opt_detector.device)
    anc_sn_cuda = torch.unsqueeze(anc_sn1, dim=0).to(opt_detector.device)
    anc_node_cuda = torch.unsqueeze(anc_node1, dim=0).to(opt_detector.device)


    #print(anc_pc_cuda.size(), anc_sn_cuda.size(), anc_node_cuda.size())
    tic = time.time()
    anc_keypoints, anc_sigmas = model_detector.run_model(anc_pc_cuda, anc_sn_cuda, anc_node_cuda)
    anc_keypoints_np = anc_keypoints.detach().permute(0, 2, 1).contiguous().cpu().numpy()  # BxMx3
    anc_sigmas_np = anc_sigmas.detach().cpu().numpy()  # BxM
        
    
    toc = 1000 * (time.time() - tic)

        
    for b in range(anc_pc.size(0)):
        frame_pc_np = np.transpose(anc_pc[b].detach().numpy())  # Nx3

        frame_keypoint_np = anc_keypoints_np[b]
        frame_sigma_np = anc_sigmas_np[b]

        # nms
        keypoints, frame_sigma_np = nms(frame_keypoint_np, frame_sigma_np, NMS_radius=NMS_radius)

    
    
    #print("SIFT Computation took {:.0f} [ms]".format(toc), " count :", len(keypoints))     
    point_cloud_kps = o3d.geometry.PointCloud()
    point_cloud_kps.points = o3d.utility.Vector3dVector(keypoints) 
    
        
    return point_cloud_kps, toc  


  

In [9]:
from evaluation.kitti_test_loader import  KittiTestLoader_Odometry, KittiTestLoader_Odometry_Original
import torch

def get_kitti_sample(start_index=0):
    numpy_folder = '/home/pranayspeed/Work/git_repos/TSF_datasets/kitti_odometry_numpy'    
    
    test_txt_folder = '/home/pranayspeed/Work/git_repos/TSF_datasets/kitti_odometry_bin'

    testset = KittiTestLoader_Odometry(test_txt_folder, numpy_folder, opt_detector)

    # test_txt_folder = '/home/pranayspeed/Work/git_repos/data_odometry_velodyne_part/sequences'
    # testset = KittiTestLoader_Odometry_Original(test_txt_folder, numpy_folder, opt_detector)
    
    testloader = torch.utils.data.DataLoader(testset, batch_size=opt_detector.batch_size,
                                                shuffle=False, num_workers=opt_detector.nThreads, pin_memory=False)
    
    i, data = next(enumerate(testloader, start_index))
    anc_pc, anc_sn, anc_node, seq, anc_idx, gt_pose = data
    for b in range(anc_pc.size(0)):
        frame_pc_np = np.transpose(anc_pc[b].detach().numpy())  # Nx3
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(frame_pc_np) 
        
        frame_normals_np = np.transpose(anc_sn[b].detach().numpy())  # Nx4
        
        pcd.normals = o3d.utility.Vector3dVector(frame_normals_np[:,:3]) 
        #print("seq frame id : ", i)
        return pcd , (anc_pc, anc_sn, anc_node)

from evaluation.oxford_test_loader import OxfordTestLoader   
def get_oxford_sample(start_index=0):
    root_folder = '/home/pranayspeed/Work/git_repos/TSF_datasets/oxford'
    # testset = OxfordDataLoader_Odometry(root_folder, self.opt_detector)
    
    testset = OxfordTestLoader(root_folder, opt_detector)
    
    testloader = torch.utils.data.DataLoader(testset, batch_size=opt_detector.batch_size,
                                                shuffle=False, num_workers=opt_detector.nThreads, pin_memory=False)
    
    i, data = next(enumerate(testloader, start_index))
    anc_pc, anc_sn, anc_node, seq_id= data
    for b in range(anc_pc.size(0)):
        frame_pc_np = np.transpose(anc_pc[b].detach().numpy())  # Nx3
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(frame_pc_np) 
        # print("seq frame id : ", i)
        # return pcd

        frame_normals_np = np.transpose(anc_sn[b].detach().numpy())  # Nx4
        
        pcd.normals = o3d.utility.Vector3dVector(frame_normals_np[:,:3]) 
        #print("seq frame id : ", i)
        return pcd , (anc_pc, anc_sn, anc_node)
    

from evaluation.kitti_test_loader import SyntheticDataLoader   
def get_synth_sample(start_index=0):
    root_folder = '/home/pranayspeed/Work/git_repos/TSF_datasets/mai_data'
    testset = SyntheticDataLoader(root_folder, opt_detector)
    
    testloader = torch.utils.data.DataLoader(testset, batch_size=opt_detector.batch_size,
                                                shuffle=False, num_workers=opt_detector.nThreads, pin_memory=False)
    
    i, data = next(enumerate(testloader, start_index))
    anc_pc, anc_sn, anc_node, seq, anc_idx, gt_pose = data
    for b in range(anc_pc.size(0)):
        frame_pc_np = np.transpose(anc_pc[b].detach().numpy())  # Nx3
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(frame_pc_np) 
        # print("seq frame id : ", i)
        # return pcd

        frame_normals_np = np.transpose(anc_sn[b].detach().numpy())  # Nx4
        
        pcd.normals = o3d.utility.Vector3dVector(frame_normals_np[:,:3]) 
        #print("seq frame id : ", i)
        return pcd , (anc_pc, anc_sn, anc_node)




from evaluation.kitti_test_loader import ParisLucoDataLoader   
def get_luco_sample(start_index=0):
    root_folder = '/home/pranayspeed/Work/git_repos/TSF_datasets/mai_data'
    testset = ParisLucoDataLoader(root_folder, opt_detector)
    
    testloader = torch.utils.data.DataLoader(testset, batch_size=opt_detector.batch_size,
                                                shuffle=False, num_workers=opt_detector.nThreads, pin_memory=False)
    
    i, data = next(enumerate(testloader, start_index))
    anc_pc, anc_sn, anc_node, seq, anc_idx, gt_pose = data
    for b in range(anc_pc.size(0)):
        frame_pc_np = np.transpose(anc_pc[b].detach().numpy())  # Nx3
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(frame_pc_np) 
        # print("seq frame id : ", i)
        # return pcd

        frame_normals_np = np.transpose(anc_sn[b].detach().numpy())  # Nx4
        
        pcd.normals = o3d.utility.Vector3dVector(frame_normals_np[:,:3]) 
        #print("seq frame id : ", i)
        return pcd , (anc_pc, anc_sn, anc_node)


In [10]:
# bunny_mesh = o3d.data.BunnyMesh()

# mesh = o3d.io.read_triangle_mesh(bunny_mesh.path)
# mesh.compute_vertex_normals()

# mesh_pcd =mesh.sample_points_uniformly(500)
## o3d.visualization.draw_geometries([mesh, mesh_pcd])
# test_transf = get_random_transform()
# orig_pcd , pcd_transf = get_transformed_point_cloud(mesh_pcd.points, test_transf)
# orig_pcd_kps, orig_extract_time = extract_keypoints_ISS(orig_pcd)
# pcd_transf_kps, pcd_transf_time = extract_keypoints_ISS(pcd_transf)

# pcd_transf_kps_inv, _ = compute_inverse_transform_pcd(pcd_transf_kps, test_transf)
# compute_repeatablity(orig_pcd_kps, pcd_transf_kps_inv)
# compute_repeatablity(orig_pcd_kps, pcd_transf_kps)

In [78]:
def get_delta_translation_transform():
    point_cloud = o3d.geometry.PointCloud()
    euler_angles = R.random().as_euler('zxy', degrees=False) #random radian rotation


    list_Size = 3
    # random float from 1 to 99.9
    integer_list = random.sample(range(1, 20), list_Size)
    float_list = [x/100 for x in integer_list]
              
    T = np.eye(4)
    #T[:3, :3] = point_cloud.get_rotation_matrix_from_xyz(euler_angles)
    T[0, 3] = float_list[0]
    T[1, 3] = float_list[1]
    T[2, 3] = float_list[2]
    return T

def get_delta_rotation_transform():
    point_cloud = o3d.geometry.PointCloud()
    
    one_degree = 0.0174533
    
    euler_angles = R.random().as_euler('xyz', degrees=False) #random radian rotation

    
    list_Size = 3
    # random float from 1 to 99.9
    integer_list = random.sample(range(1, 5), list_Size)
    float_list = [x*one_degree for x in integer_list]
    
    
    euler_angles = float_list
              
    T = np.eye(4)
    T[:3, :3] = point_cloud.get_rotation_matrix_from_xyz(euler_angles)
    # T[0, 3] = float_list[0]
    # T[1, 3] = float_list[1]
    # T[2, 3] = float_list[2]
    return T

# def get_delta_transform(delta_trans=5.0, delta_rot=5.0):
#     point_cloud = o3d.geometry.PointCloud()
    
#     one_degree = 0.0174533
    
#     euler_angles = R.random().as_euler('xyz', degrees=False) #random radian rotation

    
#     list_Size = 3
#     # random float from 1 to 99.9
#     integer_list1 = random.sample(range(1, 5), list_Size)
#     euler_angles = [x*one_degree*(delta_rot/5) for x in integer_list1]
    

#     # random float from 1 to 99.9
#     integer_list = random.sample(range(1, 20), list_Size)
#     float_list = [x/10*(delta_trans/5) for x in integer_list]
            
              
#     T = np.eye(4)
#     T[:3, :3] = point_cloud.get_rotation_matrix_from_xyz(euler_angles)
#     T[0, 3] = float_list[0]
#     T[1, 3] = float_list[1]
#     T[2, 3] = float_list[2]
#     return T

def generate_k_random_normalized(delta, k=3, low=0.05):
    if delta ==0:
        return np.zeros(k)
    a = np.random.rand(k)
    a = (a/a.sum()*(delta-low*k))
    weights = a+low

    # checking that the sum is 1
    assert np.isclose(weights.sum(), delta)
    return weights

def get_delta_transform(delta_trans=1.0, delta_rot=1.0):
    point_cloud = o3d.geometry.PointCloud()
    
    one_degree = 0.0174533
    
    euler_angles = R.random().as_euler('xyz', degrees=False) #random radian rotation


    euler_angles = generate_k_random_normalized(delta_rot, 3,0.05)*one_degree
    

    # random float from 1 to 99.9
    #integer_list = random.sample(range(1, 20), list_Size)
    float_list = generate_k_random_normalized(delta_trans, 3, 0.05)
            
              
    T = np.eye(4)
    T[:3, :3] = point_cloud.get_rotation_matrix_from_xyz(euler_angles)
    T[0, 3] = float_list[0]
    T[1, 3] = float_list[1]
    T[2, 3] = float_list[2]
    return T


In [79]:


def compute_transform_invariant_repeatiblity(input_orig_pcd, keypoint_extraction_fn, transf=None, target_pcd=None, search_radius=0.5):
    
    input_orig_pcd_np, (anc_pc, anc_sn, anc_node)=input_orig_pcd
    
    test_transf = transf
    if test_transf is None:
        test_transf = get_delta_transform()
    

        
    orig_pcd , pcd_transf = get_transformed_point_cloud(input_orig_pcd_np, test_transf)
    
    farthest_sampler = FarthestSampler()
    
    orig_pcd_kps, orig_extract_time = keypoint_extraction_fn(orig_pcd, anc_pc, anc_sn, anc_node, farthest_sampler)
    
    if target_pcd is not None:
        pcd_transf, (anc_pc, anc_sn, anc_node)=target_pcd
    
    pcd_transf_kps, pcd_transf_time = keypoint_extraction_fn(pcd_transf, anc_pc, anc_sn, anc_node, farthest_sampler) 
    
    if  len(orig_pcd_kps.points) ==0 or len(pcd_transf_kps.points)==0:
        print("Keypoint Extraction failed")
        return 0.0
    pcd_transf_kps_inv, _ = compute_inverse_transform_pcd(pcd_transf_kps, test_transf)
    
    
    
    repeatiblity_val, points_cnt, matches = compute_repeatablity(orig_pcd_kps, pcd_transf_kps_inv, search_radius)
    return repeatiblity_val, orig_pcd, orig_pcd_kps, pcd_transf_kps_inv, points_cnt, matches
      

In [80]:
tst_transf = get_delta_rotation_transform()
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data, points_cnt, matches  = compute_transform_invariant_repeatiblity(get_kitti_sample(2),extract_keypoints_usip, tst_transf)
print(points_cnt, matches, "\n",tst_transf,"\n", rep_val)
tst_transf = get_delta_translation_transform()
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data, points_cnt, matches  = compute_transform_invariant_repeatiblity(get_kitti_sample(2),extract_keypoints_usip, tst_transf)
print(points_cnt, matches, "\n",tst_transf,"\n", rep_val)

tst_transf = get_delta_transform(1.0,0.0)
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data, points_cnt, matches  = compute_transform_invariant_repeatiblity(get_kitti_sample(2),extract_keypoints_usip, tst_transf)
print(points_cnt, matches, "\n",tst_transf,"\n", rep_val)

tst_transf = get_delta_transform(0.0,1.0)
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data, points_cnt, matches  = compute_transform_invariant_repeatiblity(get_kitti_sample(2),extract_keypoints_usip, tst_transf)
print(points_cnt, matches, "\n",tst_transf,"\n", rep_val)

512 242 
 [[ 0.99923861 -0.0348942   0.01745241  0.        ]
 [ 0.03576452  0.99798932 -0.05232801  0.        ]
 [-0.01559138  0.05291234  0.99847744  0.        ]
 [ 0.          0.          0.          1.        ]] 
 0.47265625
512 296 
 [[1.   0.   0.   0.14]
 [0.   1.   0.   0.12]
 [0.   0.   1.   0.09]
 [0.   0.   0.   1.  ]] 
 0.578125
512 261 
 [[1.         0.         0.         0.2508814 ]
 [0.         1.         0.         0.42390589]
 [0.         0.         1.         0.32521271]
 [0.         0.         0.         1.        ]] 
 0.509765625
512 277 
 [[ 0.99996447 -0.00288562  0.00792034  0.        ]
 [ 0.0029383   0.99997359 -0.00664691  0.        ]
 [-0.00790095  0.00666994  0.99994654  0.        ]
 [ 0.          0.          0.          1.        ]] 
 0.541015625


In [62]:
def generate_k_random_normalized(k=3, low=0.05):

    a = np.random.rand(k)
    a = (a/a.sum()*(1-low*k))
    weights = a+low

    # checking that the sum is 1
    assert np.isclose(weights.sum(), 1)
    return weights

In [48]:
generate_k_random_normalized()

array([0.46020389, 0.21337133, 0.32642478])

In [13]:
tst_transf = get_random_transform()
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_transform_invariant_repeatiblity(get_kitti_sample(2),extract_keypoints_Harris, tst_transf)

ValueError: too many values to unpack (expected 4)

In [21]:
orig_kps_data.paint_uniform_color((0, 1, 0))
transf_kps_data.paint_uniform_color((1, 0, 0))
orig_pcd_data.paint_uniform_color((0.5, 0.5, 0.5))
o3d.visualization.draw_geometries([orig_pcd_data, orig_kps_data, transf_kps_data])

In [13]:
tst_transf = get_random_transform()
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_transform_invariant_repeatiblity(get_luco_sample(2),extract_keypoints_usip, tst_transf)

In [13]:
tst_transf = get_random_transform()
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_transform_invariant_repeatiblity(get_luco_sample(2),extract_keypoints_ISS, tst_transf)

curvature  (63033, 1)
(63033, 4)
curvature  (63039, 1)
(63039, 4)
seq frame id :  2
ISS Computation took 149 [ms]
ISS Computation took 157 [ms]
Matches :  2438 / 2998
Repeatabilty:  0.8132088058705804


In [14]:
orig_kps_data.paint_uniform_color((0, 1, 0))
transf_kps_data.paint_uniform_color((1, 0, 0))
orig_pcd_data.paint_uniform_color((0.5, 0.5, 0.5))
o3d.visualization.draw_geometries([orig_pcd_data, orig_kps_data, transf_kps_data])

In [131]:
tst_transf = get_random_transform()
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_transform_invariant_repeatiblity(get_kitti_sample(),extract_keypoints_ISS, tst_transf)

seq frame id :  0
ISS Computation took 49 [ms]
ISS Computation took 43 [ms]
Matches :  522 / 672
Repeatabilty:  0.7767857142857143


In [132]:
orig_kps_data.paint_uniform_color((0, 1, 0))
transf_kps_data.paint_uniform_color((1, 0, 0))
orig_pcd_data.paint_uniform_color((0.5, 0.5, 0.5))
o3d.visualization.draw_geometries([orig_pcd_data, orig_kps_data, transf_kps_data])

In [52]:
tst_transf = get_random_transform()
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_transform_invariant_repeatiblity(get_kitti_sample(),extract_keypoints_SIFT, tst_transf)

seq frame id :  0
SIFT Computation took 160 [ms]  count : 192
SIFT Computation took 187 [ms]  count : 307
Matches :  44 / 192
Repeatabilty:  0.22916666666666666


In [15]:
orig_kps_data.paint_uniform_color((0, 1, 0))
transf_kps_data.paint_uniform_color((1, 0, 0))
orig_pcd_data.paint_uniform_color((0.5, 0.5, 0.5))
o3d.visualization.draw_geometries([orig_pcd_data, orig_kps_data, transf_kps_data])

In [53]:
compute_transform_invariant_repeatiblity(get_kitti_sample(),extract_keypoints_ISS, tst_transf)

seq frame id :  0
ISS Computation took 46 [ms]
ISS Computation took 38 [ms]
Matches :  540 / 682
Repeatabilty:  0.7917888563049853


(0.7917888563049853,
 PointCloud with 16384 points.,
 PointCloud with 682 points.,
 PointCloud with 540 points.)

In [19]:
orig_kps_data.paint_uniform_color((0, 1, 0))
transf_kps_data.paint_uniform_color((1, 0, 0))
orig_pcd_data.paint_uniform_color((0.5, 0.5, 0.5))
o3d.visualization.draw_geometries([orig_pcd_data, orig_kps_data, transf_kps_data])

In [54]:
tst_transf = get_random_transform()
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_transform_invariant_repeatiblity(get_oxford_sample(),extract_keypoints_ISS, tst_transf)


seq frame id :  0
ISS Computation took 46 [ms]
ISS Computation took 39 [ms]
Matches :  366 / 396
Repeatabilty:  0.9242424242424242


In [ ]:
orig_kps_data.paint_uniform_color((0, 1, 0))
transf_kps_data.paint_uniform_color((1, 0, 0))
orig_pcd_data.paint_uniform_color((0.5, 0.5, 0.5))
o3d.visualization.draw_geometries([orig_pcd_data, orig_kps_data, transf_kps_data])

## Robust to Noise

In [ ]:
from walle.core import RotationMatrix


def gen_data(N=100, frac=0.1):
  # create a random rigid transform
  transform = np.eye(4)
  transform[:3, :3] = RotationMatrix.random()
  transform[:3, 3] = 2 * np.random.randn(3) + 1

  # create a random source point cloud
  src_pc = 5 * np.random.randn(N, 3) + 2
  dst_pc = Procrustes.transform_xyz(src_pc, transform)

  # corrupt
  rand_corrupt = np.random.choice(np.arange(len(src_pc)), replace=False, size=int(frac*N))
  dst_pc[rand_corrupt] += np.random.uniform(-10, 10, (int(frac*N), 3))

  return src_pc, dst_pc, transform, rand_corrupt

In [14]:
def apply_noise(pcd, mu, sigma):
    noisy_pcd = copy.deepcopy(pcd)
    points = np.asarray(noisy_pcd.points)
    points += np.random.normal(mu, sigma, size=points.shape)
    noisy_pcd.points = o3d.utility.Vector3dVector(points)
    return noisy_pcd

def compute_noise_invariant_repeatiblity(input_orig_pcd, keypoint_extraction_fn, noise_mag=(0.0, 0.01), target_pcd=None, search_radius=0.5):
    
    input_orig_pcd_np, (anc_pc, anc_sn, anc_node)=input_orig_pcd
    
    # test_transf = transf
    # if test_transf is None:
    #     test_transf = get_random_transform()
    
    mu, sigma = noise_mag # 0, 0.1  # mean and standard deviation
    pcd_transf = apply_noise(input_orig_pcd_np, mu, sigma)

    orig_pcd = copy.deepcopy(input_orig_pcd_np)    
    #orig_pcd , pcd_transf = get_transformed_point_cloud(input_orig_pcd_np, test_transf)
    
    farthest_sampler = FarthestSampler()
    
    orig_pcd_kps, orig_extract_time = keypoint_extraction_fn(orig_pcd, anc_pc, anc_sn, anc_node, farthest_sampler)
    
    if target_pcd is not None:
        pcd_transf, (anc_pc, anc_sn, anc_node)=target_pcd
    
    pcd_transf_kps, pcd_transf_time = keypoint_extraction_fn(pcd_transf, anc_pc, anc_sn, anc_node, farthest_sampler) 
    
    if  len(orig_pcd_kps.points) ==0 or len(pcd_transf_kps.points)==0:
        print("Keypoint Extraction failed")
        return 0.0
    #pcd_transf_kps_inv, _ = compute_inverse_transform_pcd(pcd_transf_kps, test_transf)
    
    pcd_transf_kps_inv = copy.deepcopy(pcd_transf_kps) 
    
    repeatiblity_val, points_cnt, matches = compute_repeatablity(orig_pcd_kps, pcd_transf_kps_inv, search_radius)
    return repeatiblity_val, orig_pcd, orig_pcd_kps, pcd_transf_kps_inv, points_cnt, matches

In [60]:
mu, sigma =  0.00001, 0.00001  
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_noise_invariant_repeatiblity(get_oxford_sample(),extract_keypoints_ISS, (mu, sigma))

seq frame id :  0
ISS Computation took 49 [ms]
ISS Computation took 51 [ms]
Matches :  372 / 414
Repeatabilty:  0.8985507246376812


In [61]:
orig_kps_data.paint_uniform_color((0, 1, 0))
transf_kps_data.paint_uniform_color((1, 0, 0))
orig_pcd_data.paint_uniform_color((0.5, 0.5, 0.5))
o3d.visualization.draw_geometries([orig_pcd_data, orig_kps_data, transf_kps_data])

In [72]:
mu, sigma =  0.0, 0.00001  
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_noise_invariant_repeatiblity(get_oxford_sample(),extract_keypoints_SIFT, (mu, sigma))

seq frame id :  0
SIFT Computation took 176 [ms]  count : 94
SIFT Computation took 180 [ms]  count : 94
Matches :  69 / 94
Repeatabilty:  0.7340425531914894


In [14]:
mu, sigma =  0, 0.001  
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_noise_invariant_repeatiblity(get_oxford_sample(),extract_keypoints_usip, (mu, sigma))

seq frame id :  0
SIFT Computation took 266 [ms]  count : 512
SIFT Computation took 259 [ms]  count : 512
Matches :  260 / 512
Repeatabilty:  0.5078125


## Robust to Resolution

In [81]:
def apply_downsampling(pcd, voxel_size):
    noisy_pcd = copy.deepcopy(pcd)
    noisy_pcd.voxel_down_sample(voxel_size)
    return noisy_pcd

def compute_resolution_invariant_repeatiblity(input_orig_pcd, keypoint_extraction_fn, voxel_size=0.5, target_pcd=None, search_radius=0.5):
    
    input_orig_pcd_np, (anc_pc, anc_sn, anc_node)=input_orig_pcd
    
    # test_transf = transf
    # if test_transf is None:
    #     test_transf = get_random_transform()
    
    pcd_transf = apply_downsampling(input_orig_pcd_np, voxel_size)

    orig_pcd = copy.deepcopy(input_orig_pcd_np)    
    #orig_pcd , pcd_transf = get_transformed_point_cloud(input_orig_pcd_np, test_transf)
    
    farthest_sampler = FarthestSampler()
    
    orig_pcd_kps, orig_extract_time = keypoint_extraction_fn(orig_pcd, anc_pc, anc_sn, anc_node, farthest_sampler)
    
    if target_pcd is not None:
        pcd_transf, (anc_pc, anc_sn, anc_node)=target_pcd
    
    pcd_transf_kps, pcd_transf_time = keypoint_extraction_fn(pcd_transf, anc_pc, anc_sn, anc_node, farthest_sampler) 
    
    if  len(orig_pcd_kps.points) ==0 or len(pcd_transf_kps.points)==0:
        print("Keypoint Extraction failed")
        return 0.0
    #pcd_transf_kps_inv, _ = compute_inverse_transform_pcd(pcd_transf_kps, test_transf)
    
    pcd_transf_kps_inv = copy.deepcopy(pcd_transf_kps) 
    
    repeatiblity_val, points_cnt, matches = compute_repeatablity(orig_pcd_kps, pcd_transf_kps_inv, search_radius)
    return repeatiblity_val, orig_pcd, orig_pcd_kps, pcd_transf_kps_inv, points_cnt, matches

In [16]:
voxel_size=0.5
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data, pt_cnt, match_cnt = compute_resolution_invariant_repeatiblity(get_oxford_sample(),extract_keypoints_ISS,voxel_size)

In [16]:
voxel_size=0.1
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_resolution_invariant_repeatiblity(get_oxford_sample(),extract_keypoints_SIFT,voxel_size)

seq frame id :  0
SIFT Computation took 181 [ms]  count : 87
SIFT Computation took 174 [ms]  count : 87
Matches :  67 / 87
Repeatabilty:  0.7701149425287356


In [17]:
voxel_size=0.5
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_resolution_invariant_repeatiblity(get_oxford_sample(),extract_keypoints_usip,voxel_size)

seq frame id :  0
SIFT Computation took 263 [ms]  count : 512
SIFT Computation took 260 [ms]  count : 512
Matches :  265 / 512
Repeatabilty:  0.517578125


In [20]:
orig_kps_data.paint_uniform_color((0, 1, 0))
transf_kps_data.paint_uniform_color((1, 0, 0))
orig_pcd_data.paint_uniform_color((0.5, 0.5, 0.5))
o3d.visualization.draw_geometries([orig_pcd_data, orig_kps_data, transf_kps_data])

## Repeatability Experiment

In [164]:
datasets = {
    "kitti": get_kitti_sample,
    "oxford": get_oxford_sample,
    "luco": get_luco_sample
}


keypoint_detectors = {
    "Harris": extract_keypoints_Harris,
    "ISS": extract_keypoints_ISS,
    "SIFT": extract_keypoints_SIFT,
    "USIP": extract_keypoints_usip
}




In [165]:
def run_transformation_invariant_repeatiblity(keypoint_detectors, datasets):
    results = {}
    
    translation_list = [trans_val_id for trans_val_id in range(0,20)]
    
    curr_transf_list = [get_delta_transform(delta_trans=1.0, delta_rot=i) for i in range(0,20)]
                    
    for dataset in datasets:
        curr_dataset_fn = datasets[dataset]
          
        print("current dataset:", dataset)
        repeat_list = {}
        matches_list = {}
        points_count_list = {}
        for dataset_seq_index in range(5):     
            curr_data = curr_dataset_fn(dataset_seq_index)
            for keypoint_detect in keypoint_detectors:
                curr_detector_fn = keypoint_detectors[keypoint_detect]
                        
                if keypoint_detect not in repeat_list:
                    repeat_list[keypoint_detect] = {}
                    matches_list[keypoint_detect] = {}
                    points_count_list[keypoint_detect] = {}
                # if dataset_seq_index not in repeat_list[keypoint_detect]:
                #     repeat_list[keypoint_detect][dataset_seq_index] ={}   
                for delta_val in translation_list:
                    if delta_val not in repeat_list[keypoint_detect]:
                        repeat_list[keypoint_detect][delta_val]=[]
                        matches_list[keypoint_detect][delta_val]=[]
                        points_count_list[keypoint_detect][delta_val]=[]
                    
                print("current detector:", keypoint_detect)
                
                for delta_val in translation_list:
                    curr_transf = curr_transf_list[delta_val]
                    rep_val, orig_pcd_data, orig_kps_data, transf_kps_data, points_count, matches = compute_transform_invariant_repeatiblity(curr_data,curr_detector_fn, curr_transf)

                    repeat_list[keypoint_detect][delta_val].append(rep_val)
                    matches_list[keypoint_detect][delta_val].append(matches)
                    points_count_list[keypoint_detect][delta_val].append(points_count)



        for keypoint_detect in keypoint_detectors:  
            if keypoint_detect not in results:
                results[keypoint_detect] = {}
            if dataset not in results[keypoint_detect]:
                results[keypoint_detect][dataset]={}    
            for delta_val in translation_list:
                
                results[keypoint_detect][dataset][delta_val] ={}
                
                repeatablity_val = np.array(repeat_list[keypoint_detect][delta_val]).mean()
                avg_keypoints_detected = np.array(points_count_list[keypoint_detect][delta_val]).mean()
                avg_keypoints_matched = np.array(matches_list[keypoint_detect][delta_val]).mean()
                print(keypoint_detect, dataset, delta_val, "->",repeatablity_val, avg_keypoints_detected, avg_keypoints_matched)

                
                results[keypoint_detect][dataset][delta_val]["repeatability"]= repeatablity_val
                results[keypoint_detect][dataset][delta_val]["matched"]= avg_keypoints_matched
                results[keypoint_detect][dataset][delta_val]["detected"]= avg_keypoints_detected
    return results

In [166]:
def run_transformation_invariant_translation_only_repeatiblity(keypoint_detectors, datasets):
    results = {}
    
    translation_list = [trans_val_id for trans_val_id in range(0,20)]
    
    curr_transf_list = [get_delta_transform(delta_trans=i, delta_rot=0.0) for i in range(0,20)]
                    
    for dataset in datasets:
        curr_dataset_fn = datasets[dataset]
          
        print("current dataset:", dataset)
        repeat_list = {}
        matches_list = {}
        points_count_list = {}
        for dataset_seq_index in range(5):     
            curr_data = curr_dataset_fn(dataset_seq_index)
            for keypoint_detect in keypoint_detectors:
                curr_detector_fn = keypoint_detectors[keypoint_detect]
                        
                if keypoint_detect not in repeat_list:
                    repeat_list[keypoint_detect] = {}
                    matches_list[keypoint_detect] = {}
                    points_count_list[keypoint_detect] = {}
                # if dataset_seq_index not in repeat_list[keypoint_detect]:
                #     repeat_list[keypoint_detect][dataset_seq_index] ={}   
                for delta_val in translation_list:
                    if delta_val not in repeat_list[keypoint_detect]:
                        repeat_list[keypoint_detect][delta_val]=[]
                        matches_list[keypoint_detect][delta_val]=[]
                        points_count_list[keypoint_detect][delta_val]=[]
                    
                print("current detector:", keypoint_detect)
                
                for delta_val in translation_list:
                    curr_transf = curr_transf_list[delta_val]
                    rep_val, orig_pcd_data, orig_kps_data, transf_kps_data, points_count, matches = compute_transform_invariant_repeatiblity(curr_data,curr_detector_fn, curr_transf)

                    repeat_list[keypoint_detect][delta_val].append(rep_val)
                    matches_list[keypoint_detect][delta_val].append(matches)
                    points_count_list[keypoint_detect][delta_val].append(points_count)



        for keypoint_detect in keypoint_detectors:  
            if keypoint_detect not in results:
                results[keypoint_detect] = {}
            if dataset not in results[keypoint_detect]:
                results[keypoint_detect][dataset]={}    
            for delta_val in translation_list:
                
                results[keypoint_detect][dataset][delta_val] ={}
                
                repeatablity_val = np.array(repeat_list[keypoint_detect][delta_val]).mean()
                avg_keypoints_detected = np.array(points_count_list[keypoint_detect][delta_val]).mean()
                avg_keypoints_matched = np.array(matches_list[keypoint_detect][delta_val]).mean()
                print(keypoint_detect, dataset, delta_val, "->",repeatablity_val, avg_keypoints_detected, avg_keypoints_matched)

                
                results[keypoint_detect][dataset][delta_val]["repeatability"]= repeatablity_val
                results[keypoint_detect][dataset][delta_val]["matched"]= avg_keypoints_matched
                results[keypoint_detect][dataset][delta_val]["detected"]= avg_keypoints_detected
    return results
        

In [167]:
def run_transformation_invariant_rotation_only_repeatiblity(keypoint_detectors, datasets):
    results = {}
    
    translation_list = [trans_val_id for trans_val_id in range(0,20)]
    
    curr_transf_list = [get_delta_transform(delta_trans=0.0, delta_rot=i) for i in range(0,20)]
                    
    for dataset in datasets:
        curr_dataset_fn = datasets[dataset]
          
        print("current dataset:", dataset)
        repeat_list = {}
        matches_list = {}
        points_count_list = {}
        for dataset_seq_index in range(5):     
            curr_data = curr_dataset_fn(dataset_seq_index)
            for keypoint_detect in keypoint_detectors:
                curr_detector_fn = keypoint_detectors[keypoint_detect]
                        
                if keypoint_detect not in repeat_list:
                    repeat_list[keypoint_detect] = {}
                    matches_list[keypoint_detect] = {}
                    points_count_list[keypoint_detect] = {}
                # if dataset_seq_index not in repeat_list[keypoint_detect]:
                #     repeat_list[keypoint_detect][dataset_seq_index] ={}   
                for delta_val in translation_list:
                    if delta_val not in repeat_list[keypoint_detect]:
                        repeat_list[keypoint_detect][delta_val]=[]
                        matches_list[keypoint_detect][delta_val]=[]
                        points_count_list[keypoint_detect][delta_val]=[]
                    
                print("current detector:", keypoint_detect)
                
                for delta_val in translation_list:
                    curr_transf = curr_transf_list[delta_val]
                    rep_val, orig_pcd_data, orig_kps_data, transf_kps_data, points_count, matches = compute_transform_invariant_repeatiblity(curr_data,curr_detector_fn, curr_transf)

                    repeat_list[keypoint_detect][delta_val].append(rep_val)
                    matches_list[keypoint_detect][delta_val].append(matches)
                    points_count_list[keypoint_detect][delta_val].append(points_count)



        for keypoint_detect in keypoint_detectors:  
            if keypoint_detect not in results:
                results[keypoint_detect] = {}
            if dataset not in results[keypoint_detect]:
                results[keypoint_detect][dataset]={}    
            for delta_val in translation_list:
                
                results[keypoint_detect][dataset][delta_val] ={}
                
                repeatablity_val = np.array(repeat_list[keypoint_detect][delta_val]).mean()
                avg_keypoints_detected = np.array(points_count_list[keypoint_detect][delta_val]).mean()
                avg_keypoints_matched = np.array(matches_list[keypoint_detect][delta_val]).mean()
                print(keypoint_detect, dataset, delta_val, "->",repeatablity_val, avg_keypoints_detected, avg_keypoints_matched)

                
                results[keypoint_detect][dataset][delta_val]["repeatability"]= repeatablity_val
                results[keypoint_detect][dataset][delta_val]["matched"]= avg_keypoints_matched
                results[keypoint_detect][dataset][delta_val]["detected"]= avg_keypoints_detected
    return results

In [168]:
res_1_rotation_only = run_transformation_invariant_rotation_only_repeatiblity(keypoint_detectors, datasets)

current dataset: kitti
current detector: Harris
current detector: ISS
current detector: SIFT
current detector: USIP
current detector: Harris
current detector: ISS
current detector: SIFT
current detector: USIP
current detector: Harris
current detector: ISS
current detector: SIFT
current detector: USIP
current detector: Harris
current detector: ISS
current detector: SIFT
current detector: USIP
current detector: Harris
current detector: ISS
current detector: SIFT
current detector: USIP
Harris kitti 0 -> 1.0 213.2 213.2
Harris kitti 1 -> 0.9849201110548723 213.2 210.0
Harris kitti 2 -> 0.9850859281396845 213.2 210.0
Harris kitti 3 -> 0.9859633002702536 213.2 210.2
Harris kitti 4 -> 0.987753456689566 213.2 210.6
Harris kitti 5 -> 0.989770827261087 213.2 211.0
Harris kitti 6 -> 0.9812073964709193 213.2 209.2
Harris kitti 7 -> 0.9821775736831789 213.2 209.4
Harris kitti 8 -> 0.9822367830638022 213.2 209.4
Harris kitti 9 -> 0.9859317013878766 213.2 210.2
Harris kitti 10 -> 0.9841415449685641 2

In [169]:
df_rep_transf_rot_only = pd.DataFrame.from_dict(res_1_rotation_only)
df_rep_transf_rot_only.head()
df_rep_transf_rot_only.to_json("Transformation_invariant_Repeatablity_rotation_only.json")
df_rep_transf_rot_only.to_latex("Transformation_invariant_Repeatablity_rotation_only.latex")
df_rep_transf_rot_only.to_html("Transformation_invariant_Repeatablity_rotation_only.html")

In [170]:
df_rep_transf_rot_only.head()

,Harris,ISS,SIFT,USIP
kitti,"{0: {'repeatability': 1.0, 'matched': 213.2, '...","{0: {'repeatability': 0.9982762689286092, 'mat...","{0: {'repeatability': 0.7870557963027618, 'mat...","{0: {'repeatability': 0.50859375, 'matched': 2..."
oxford,"{0: {'repeatability': 1.0, 'matched': 119.2, '...","{0: {'repeatability': 1.0, 'matched': 416.6, '...","{0: {'repeatability': 0.760975702330382, 'matc...","{0: {'repeatability': 0.534375, 'matched': 273..."
luco,"{0: {'repeatability': 1.0, 'matched': 344.0, '...","{0: {'repeatability': 0.9989993326362511, 'mat...","{0: {'repeatability': 0.7772511848341233, 'mat...","{0: {'repeatability': 0.231640625, 'matched': ..."


In [171]:

res_1_translation_only = run_transformation_invariant_translation_only_repeatiblity(keypoint_detectors, datasets)

current dataset: kitti
current detector: Harris
current detector: ISS
current detector: SIFT
current detector: USIP
current detector: Harris
current detector: ISS
current detector: SIFT
current detector: USIP
current detector: Harris
current detector: ISS
current detector: SIFT
current detector: USIP
current detector: Harris
current detector: ISS
current detector: SIFT
current detector: USIP
current detector: Harris
current detector: ISS
current detector: SIFT
current detector: USIP
Harris kitti 0 -> 1.0 215.0 215.0
Harris kitti 1 -> 0.9862730857929476 215.0 212.0
Harris kitti 2 -> 0.9882221539391989 215.0 212.4
Harris kitti 3 -> 0.9864059938205925 215.0 212.0
Harris kitti 4 -> 0.9926885952718884 215.0 213.4
Harris kitti 5 -> 0.9827053733862039 215.0 211.2
Harris kitti 6 -> 0.9844569256743412 215.0 211.6
Harris kitti 7 -> 0.9905879168312776 215.0 213.0
Harris kitti 8 -> 0.9854698603612171 215.0 211.8
Harris kitti 9 -> 0.9927690716922972 215.0 213.4
Harris kitti 10 -> 0.9890253793709294

In [172]:
res_1_translation_only

{'Harris': {'kitti': {0: {'repeatability': 1.0,
    'matched': 215.0,
    'detected': 215.0},
   1: {'repeatability': 0.9862730857929476,
    'matched': 212.0,
    'detected': 215.0},
   2: {'repeatability': 0.9882221539391989,
    'matched': 212.4,
    'detected': 215.0},
   3: {'repeatability': 0.9864059938205925,
    'matched': 212.0,
    'detected': 215.0},
   4: {'repeatability': 0.9926885952718884,
    'matched': 213.4,
    'detected': 215.0},
   5: {'repeatability': 0.9827053733862039,
    'matched': 211.2,
    'detected': 215.0},
   6: {'repeatability': 0.9844569256743412,
    'matched': 211.6,
    'detected': 215.0},
   7: {'repeatability': 0.9905879168312776,
    'matched': 213.0,
    'detected': 215.0},
   8: {'repeatability': 0.9854698603612171,
    'matched': 211.8,
    'detected': 215.0},
   9: {'repeatability': 0.9927690716922972,
    'matched': 213.4,
    'detected': 215.0},
   10: {'repeatability': 0.9890253793709294,
    'matched': 212.6,
    'detected': 215.0},
   11

In [173]:
df_rep_transf_trans_only = pd.DataFrame.from_dict(res_1_translation_only)
df_rep_transf_trans_only.head()
df_rep_transf_trans_only.to_json("Transformation_invariant_Repeatablity_Translation_only.json")
df_rep_transf_trans_only.to_latex("Transformation_invariant_Repeatablity_Translation_only.latex")
df_rep_transf_trans_only.to_html("Transformation_invariant_Repeatablity_Translation_only.html")

In [174]:
res_1 = run_transformation_invariant_repeatiblity(keypoint_detectors, datasets)

current dataset: kitti
current detector: Harris
current detector: ISS
current detector: SIFT
current detector: USIP
current detector: Harris
current detector: ISS
current detector: SIFT
current detector: USIP
current detector: Harris
current detector: ISS
current detector: SIFT
current detector: USIP
current detector: Harris
current detector: ISS
current detector: SIFT
current detector: USIP
current detector: Harris
current detector: ISS
current detector: SIFT
current detector: USIP
Harris kitti 0 -> 0.994360146560967 211.6 210.4
Harris kitti 1 -> 0.989615207174743 211.6 209.4
Harris kitti 2 -> 0.9895914838845506 211.6 209.4
Harris kitti 3 -> 0.9877360298525103 211.6 209.0
Harris kitti 4 -> 0.9915427033967458 211.6 209.8
Harris kitti 5 -> 0.9896015023282787 211.6 209.4
Harris kitti 6 -> 0.9877720088362179 211.6 209.0
Harris kitti 7 -> 0.9905584401273219 211.6 209.6
Harris kitti 8 -> 0.9885351784251644 211.6 209.2
Harris kitti 9 -> 0.9885661902982814 211.6 209.2
Harris kitti 10 -> 0.986

In [175]:
df_rep_transf = pd.DataFrame.from_dict(res_1)

In [176]:
df_rep_transf.head()

,Harris,ISS,SIFT,USIP
kitti,"{0: {'repeatability': 0.994360146560967, 'matc...","{0: {'repeatability': 0.8043786583484032, 'mat...","{0: {'repeatability': 0.4979568846202239, 'mat...","{0: {'repeatability': 0.53125, 'matched': 272...."
oxford,"{0: {'repeatability': 0.9967213114754099, 'mat...","{0: {'repeatability': 0.9177577585116634, 'mat...","{0: {'repeatability': 0.3496873643238854, 'mat...","{0: {'repeatability': 0.53125, 'matched': 272...."
luco,"{0: {'repeatability': 0.9912790697674418, 'mat...","{0: {'repeatability': 0.8260521632763247, 'mat...","{0: {'repeatability': 0.4786729857819905, 'mat...","{0: {'repeatability': 0.246875, 'matched': 126..."


In [177]:
df_rep_transf.to_csv("Transformation_invariant_Repeatablity.csv")

In [178]:
df_rep_transf.to_json("Transformation_invariant_Repeatablity.json")

In [179]:
df_rep_transf.to_html("Transformation_invariant_Repeatablity.html")

In [180]:
# ISS kitti 0.7915889284785118
# ISS oxford 0.9112456877131716
# ISS luco 0.8080709301600176
# SIFT kitti 0.4068568106435415
# SIFT oxford 0.29716109185709055
# SIFT luco 0.43364928909952594
# USIP kitti 0.1267578125
# USIP oxford 0.10751953125
# USIP luco 0.0328125

In [21]:
def run_noise_invariant_repeatiblity(keypoint_detectors, datasets):
    results = {}
    for keypoint_detect in keypoint_detectors:
        curr_detector_fn = keypoint_detectors[keypoint_detect]
        
        for dataset in datasets:
            sigma_list = [0.000001,0.00001, 0.0001, 0.001, 0.01, 0.1]
            
            repeat_list = {}
            matches_list = {}
            points_count_list = {}
            for sigma in sigma_list:
                repeat_list[sigma]=[]
                matches_list[sigma]=[]
                points_count_list[sigma]=[]
                
            curr_dataset_fn = datasets[dataset]
            
            for i in range(20):
                #mu, sigma =  0.0, 0.00001 
                mu=0.0 
                curr_data = curr_dataset_fn(i)
                for sigma in sigma_list:
                    rep_val, orig_pcd_data, orig_kps_data, transf_kps_data, points_count, matches = compute_noise_invariant_repeatiblity(curr_data,curr_detector_fn, (mu, sigma))
                    repeat_list[sigma].append(rep_val)
                    matches_list[sigma].append(matches)
                    points_count_list[sigma].append(points_count)

            if keypoint_detect not in results:
                results[keypoint_detect] = {}
            if dataset not in results[keypoint_detect]:
                results[keypoint_detect][dataset]={}
                
            for sigma in sigma_list:
                results[keypoint_detect][dataset][sigma] = {}
                
                repeatablity_val = np.array(repeat_list[sigma]).mean()
                avg_keypoints_detected = np.array(points_count_list[sigma]).mean()
                avg_keypoints_matched = np.array(matches_list[sigma]).mean()
                print(keypoint_detect, dataset, sigma, "->",repeatablity_val, avg_keypoints_detected, avg_keypoints_matched)

                
                results[keypoint_detect][dataset][sigma]["repeatability"]= repeatablity_val
                results[keypoint_detect][dataset][sigma]["matched"]= avg_keypoints_matched
                results[keypoint_detect][dataset][sigma]["detected"]= avg_keypoints_detected
    return results   

In [62]:
res_2 = run_noise_invariant_repeatiblity(keypoint_detectors, datasets)

Harris kitti 1e-06 -> 0.989537416235384 215.75 213.45
Harris kitti 1e-05 -> 0.9878142257960892 215.75 213.1
Harris kitti 0.0001 -> 0.9817073207268333 215.75 211.75
Harris kitti 0.001 -> 0.9585308240920194 215.75 206.8
Harris kitti 0.01 -> 0.8181224521810346 215.75 176.5
Harris kitti 0.1 -> 0.42520432407432807 215.75 91.75
Harris oxford 1e-06 -> 0.996394529084801 120.45 120.0
Harris oxford 1e-05 -> 0.9963684469566481 120.45 120.0
Harris oxford 0.0001 -> 0.9938942950365905 120.45 119.7
Harris oxford 0.001 -> 0.9640188067014541 120.45 116.1
Harris oxford 0.01 -> 0.8431409153959273 120.45 101.45
Harris oxford 0.1 -> 0.5330515355904366 120.45 64.15
Harris luco 1e-06 -> 0.9882267441860464 344.0 339.95
Harris luco 1e-05 -> 0.9877906976744185 344.0 339.8
Harris luco 0.0001 -> 0.9840116279069766 344.0 338.5
Harris luco 0.001 -> 0.9633720930232558 344.0 331.4
Harris luco 0.01 -> 0.8058139534883721 344.0 277.2
Harris luco 0.1 -> 0.4049418604651162 344.0 139.3
ISS kitti 1e-06 -> 0.7914952747665408

In [63]:
df_rep_noise = pd.DataFrame.from_dict(res_2)


In [64]:
df_rep_noise.to_json("Noise_invariant_Repeatablity.json")

In [65]:
df_rep_noise.to_latex("Noise_invariant_Repeatablity.latex")

In [66]:
df_rep_noise.to_html("Noise_invariant_Repeatablity.html")

In [22]:
def run_resolution_invariant_repeatiblity(keypoint_detectors, datasets):
    results = {}
    for keypoint_detect in keypoint_detectors:
        curr_detector_fn = keypoint_detectors[keypoint_detect]
        
        for dataset in datasets:
            voxel_sizes = [0.05, 0.1, 0.5, 1.0, 2.0]
            
            
            repeat_list = {}
            matches_list = {}
            points_count_list = {}
            for voxel_size in voxel_sizes:
                repeat_list[voxel_size]=[]
                matches_list[voxel_size]=[]
                points_count_list[voxel_size]=[]
                
            curr_dataset_fn = datasets[dataset]
            
            for i in range(20):
                
                curr_data = curr_dataset_fn(i)
                for voxel_size in voxel_sizes:
                    rep_val, orig_pcd_data, orig_kps_data, transf_kps_data, points_count, matches = compute_resolution_invariant_repeatiblity(curr_data,curr_detector_fn, voxel_size)
                    repeat_list[voxel_size].append(rep_val)
                    matches_list[voxel_size].append(matches)
                    points_count_list[voxel_size].append(points_count)

            if keypoint_detect not in results:
                results[keypoint_detect] = {}
            if dataset not in results[keypoint_detect]:
                results[keypoint_detect][dataset]={}
                
            for voxel_size in voxel_sizes:
                results[keypoint_detect][dataset][voxel_size] = {}
                
                repeatablity_val = np.array(repeat_list[voxel_size]).mean()
                avg_keypoints_detected = np.array(points_count_list[voxel_size]).mean()
                avg_keypoints_matched = np.array(matches_list[voxel_size]).mean()
                print(keypoint_detect, dataset, voxel_size, "->",repeatablity_val, avg_keypoints_detected, avg_keypoints_matched)

                
                results[keypoint_detect][dataset][voxel_size]["repeatability"]= repeatablity_val
                results[keypoint_detect][dataset][voxel_size]["matched"]= avg_keypoints_matched
                results[keypoint_detect][dataset][voxel_size]["detected"]= avg_keypoints_detected
    return results  

In [23]:
res_3 = run_resolution_invariant_repeatiblity(keypoint_detectors, datasets)

Harris kitti 0.05 -> 1.0 213.85 213.85
Harris kitti 0.1 -> 1.0 213.85 213.85
Harris kitti 0.5 -> 1.0 213.85 213.85
Harris kitti 1.0 -> 1.0 213.85 213.85
Harris kitti 2.0 -> 1.0 213.85 213.85
Harris oxford 0.05 -> 1.0 119.0 119.0
Harris oxford 0.1 -> 1.0 119.0 119.0
Harris oxford 0.5 -> 1.0 119.0 119.0
Harris oxford 1.0 -> 1.0 119.0 119.0
Harris oxford 2.0 -> 1.0 119.0 119.0
Harris luco 0.05 -> 1.0 344.0 344.0
Harris luco 0.1 -> 1.0 344.0 344.0
Harris luco 0.5 -> 1.0 344.0 344.0
Harris luco 1.0 -> 1.0 344.0 344.0
Harris luco 2.0 -> 1.0 344.0 344.0
ISS kitti 0.05 -> 0.9991129315348226 683.25 682.65
ISS kitti 0.1 -> 0.9989792138714364 683.05 682.35
ISS kitti 0.5 -> 0.9986849545754204 683.35 682.45
ISS kitti 1.0 -> 0.9990507577973269 682.95 682.3
ISS kitti 2.0 -> 0.9990466210167787 682.55 681.9
ISS oxford 0.05 -> 0.9990394664735076 419.0 418.6
ISS oxford 0.1 -> 0.9991741416766594 419.05 418.7
ISS oxford 0.5 -> 0.9989181420111388 419.1 418.65
ISS oxford 1.0 -> 0.9989298110164562 419.1 418.6

In [24]:
df_rep_resolution = pd.DataFrame.from_dict(res_3)

In [25]:
df_rep_resolution.to_json("Resolution_invariant_Repeatablity.json")
df_rep_resolution.to_latex("Resolution_invariant_Repeatablity.latex")
df_rep_resolution.to_html("Resolution_invariant_Repeatablity.html")

In [ ]:
tst_transf = get_random_transform()
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_transform_invariant_repeatiblity(get_oxford_sample(),extract_keypoints_ISS, tst_transf)

In [21]:
tst_transf = get_random_transform()
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_transform_invariant_repeatiblity(get_oxford_sample(),extract_keypoints_ISS, tst_transf)



########################################
mu, sigma =  0.0, 0.00001  
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_noise_invariant_repeatiblity(get_oxford_sample(),extract_keypoints_SIFT, (mu, sigma))

#######################################

voxel_size=0.5
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_resolution_invariant_repeatiblity(get_oxford_sample(),extract_keypoints_usip,voxel_size)

seq frame id :  0
ISS Computation took 48 [ms]
ISS Computation took 53 [ms]
Matches :  373 / 415
Repeatabilty:  0.8987951807228916
seq frame id :  0
SIFT Computation took 180 [ms]  count : 80
SIFT Computation took 184 [ms]  count : 80
Matches :  60 / 80
Repeatabilty:  0.75
seq frame id :  0
SIFT Computation took 265 [ms]  count : 512
SIFT Computation took 260 [ms]  count : 512
Matches :  278 / 512
Repeatabilty:  0.54296875


In [ ]:

voxel_size=0.5
rep_val, orig_pcd_data, orig_kps_data, transf_kps_data = compute_resolution_invariant_repeatiblity(get_synth_sample(),extract_keypoints_usip,voxel_size)


In [12]:
args.method="usip"
for_idx=-1

curr_transform = np.identity(4)

num_frames = 600
num_frames_to_skip_to_show = 1
# Result saver
save_dir = "result/" + args.sequence_idx
if not os.path.exists(save_dir): os.makedirs(save_dir)
ResultSaver = PoseGraphResultSaver(init_pose=curr_transform, 
                             save_gap=args.save_gap,
                             num_frames=num_frames,
                             seq_idx=args.sequence_idx,
                             save_dir=save_dir)

In [13]:
# for save the results as a video
fig_idx = 1
fig = plt.figure(fig_idx)


keypoint_extractor = KeyPointExtractors(opt_detector, method=args.method, dataset_type=args.dataset, desired_keypoint_num=args.num_icp_points)


keypoint_extraction_times=[]
keypoint_counts = []
matching_times = []
absoulte_trajectory_errors=[]
print(args.method)
next_data = keypoint_extractor.get_keypoints(seq_name)
while next_data is not None:
    
#for next_data in keypoint_extractor.get_keypoints():
    (elapsed_time, curr_scan_pts, curr_keypts, gt_pose_curr) = next_data
    next_data = keypoint_extractor.get_keypoints()
    for_idx+=1
    if for_idx ==frame_limit:
        break
    
    print(args.method +"-> Input Points count: "+str(curr_scan_pts.shape)+str(curr_keypts.shape))


    frame_pc_np_pcd = o3d.geometry.PointCloud()
    frame_pc_np_pcd.points = o3d.utility.Vector3dVector(curr_keypts) 
                    
    #frame_pc_np = o3d.geometry.PointCloud(frame_pc_np_old)
    #frame_keypoint_np, frame_desc_np = o3d.pipelines.registration.compute_sift_3d(frame_pc_np, radius=0.1)
    frame_pc_np_pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    frame_desc_np_feat = o3d.pipelines.registration.compute_fpfh_feature(frame_pc_np_pcd,
    o3d.geometry.KDTreeSearchParamHybrid(radius=50, max_nn=100))

    curr_descriptors = np.asarray(frame_desc_np_feat.data.T)
     
    
    
    curr_gt_transf = gt_pose_curr 


    keypoint_extraction_times.append(elapsed_time)
    
    keypoint_counts.append(len(curr_keypts))
    
  
    print(for_idx)

    if(for_idx == 0):
        prev_scan_pts = copy.deepcopy(curr_scan_pts)               
        prev_keypts = copy.deepcopy(curr_keypts)   
        prev_descriptors = copy.deepcopy(curr_descriptors)           
        icp_initial = np.eye(4)   
        
        #Setting first matching time to zero, as no matching required
        matching_times.append(0)             
        continue


    # get the start time
    start_time = time.time()

        
    # calc odometry                
    odom_transform = keypoint_extractor.compute_transform(prev_keypts, curr_keypts, icp_initial)

    #odom_transform = keypoint_extractor.compute_transform_with_feat(prev_keypts, curr_keypts, prev_descriptors, curr_descriptors, icp_initial)
    
    # get the end time
    end_time = time.time()
    
    # get the execution time
    elapsed_time = end_time - start_time
    matching_times.append(elapsed_time)

    # update the current (moved) pose 
    curr_transform = np.matmul(curr_transform, odom_transform)

    icp_initial = odom_transform # assumption: constant velocity model (for better next ICP converges)


    # renewal the prev information 
    prev_scan_pts = copy.deepcopy(curr_scan_pts)
    prev_keypts = copy.deepcopy(curr_keypts)
    
    prev_descriptors = copy.deepcopy(curr_descriptors)
    
    
    # save the ICP odometry pose result (no loop closure)
    ResultSaver.saveUnoptimizedPoseGraphResult(curr_transform, curr_gt_transf, for_idx) 
    if(for_idx % num_frames_to_skip_to_show == 0): 

        legends={}
        legends['KP_Extract_Time'] = keypoint_extraction_times[-1]
        legends['KP_Matching_Time'] = matching_times[-1]
        legends['KP_count'] = keypoint_counts[-1]
        
        result_dir = "results_1/"
        
        curr_ate = ResultSaver.vizCurrentTrajectory(fig_idx, seq_name, legends)
        #plt.savefig(result_dir+seq_name + ".png")
        
        absoulte_trajectory_errors.append(curr_ate)
        




usip
usip-> Input Points count: (16384, 3)(512, 3)
0
usip-> Input Points count: (16384, 3)(512, 3)
1
pred -0.049615720119258355 -0.014314498314993152 0.8172588271878212
gt -0.04690294 -0.02839928 0.8586941
usip-> Input Points count: (16384, 3)(512, 3)
2
pred -7.626134823480963 -2.610179658548029 1.763281471594091
gt -0.09374345 -0.05676064 1.716275
usip-> Input Points count: (16384, 3)(512, 3)
3
pred -7.148457419590916 -2.4993394149164 1.0504766119041706
gt -0.1406429 -0.08515762 2.574964
usip-> Input Points count: (16384, 3)(512, 3)
4
pred -4.334203760126301 -1.7694587485119646 2.3740660366733666
gt -0.1874858 -0.1135202 3.432648
usip-> Input Points count: (16384, 3)(512, 3)
5
pred 4.200409429438909 -10.327403535271763 27.258308827339437
gt -0.2343818 -0.141915 4.291335
usip-> Input Points count: (16384, 3)(512, 3)
6
pred 4.183600241473292 -10.723290364500958 28.02293308089511
gt -0.2812195 -0.1702743 5.148987
usip-> Input Points count: (16384, 3)(512, 3)
7
pred 3.892202837702183 -11.

KeyboardInterrupt: 

## Work with generated results

In [198]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import json

In [199]:
results_dir = "./repeatablity_results_5_frames/"

In [200]:
if not os.path.exists(results_dir):
    os.mkdir(results_dir)

In [201]:
results_jsons_path = [path for path in os.listdir(results_dir) if ".json" in path]

In [202]:
results_jsons_path

['Transformation_invariant_Repeatablity_rotation_only.json',
 'Transformation_invariant_Repeatablity_Translation_only.json',
 'Transformation_invariant_Repeatablity.json']

In [203]:
rep_dict = {}
rep_exp_names = []
for res_path in results_jsons_path:
    rep_name = res_path.split(".")[0]
    rep_dict[rep_name] = json.load(open(os.path.join(results_dir,res_path)))
    rep_exp_names.append(rep_name)


In [204]:
rep_dict[rep_exp_names[0]]

{'Harris': {'kitti': {'0': {'repeatability': 1.0,
    'matched': 213.2,
    'detected': 213.2},
   '1': {'repeatability': 0.9849201111, 'matched': 210.0, 'detected': 213.2},
   '2': {'repeatability': 0.9850859281, 'matched': 210.0, 'detected': 213.2},
   '3': {'repeatability': 0.9859633003, 'matched': 210.2, 'detected': 213.2},
   '4': {'repeatability': 0.9877534567, 'matched': 210.6, 'detected': 213.2},
   '5': {'repeatability': 0.9897708273, 'matched': 211.0, 'detected': 213.2},
   '6': {'repeatability': 0.9812073965, 'matched': 209.2, 'detected': 213.2},
   '7': {'repeatability': 0.9821775737, 'matched': 209.4, 'detected': 213.2},
   '8': {'repeatability': 0.9822367831, 'matched': 209.4, 'detected': 213.2},
   '9': {'repeatability': 0.9859317014, 'matched': 210.2, 'detected': 213.2},
   '10': {'repeatability': 0.984141545, 'matched': 209.8, 'detected': 213.2},
   '11': {'repeatability': 0.9849339137, 'matched': 210.0, 'detected': 213.2},
   '12': {'repeatability': 0.9887118334, 'mat

In [205]:
rep_dfs = {}
for rep_name in rep_exp_names:
    rep_dfs[rep_name] = pd.DataFrame.from_dict(rep_dict[rep_name])


In [206]:
rep_exp_names

['Transformation_invariant_Repeatablity_rotation_only',
 'Transformation_invariant_Repeatablity_Translation_only',
 'Transformation_invariant_Repeatablity']

In [207]:
extracted_results_dir = os.path.join(results_dir, "extract_results/")
if not os.path.exists(extracted_results_dir):
    os.mkdir(extracted_results_dir)

In [208]:

curr_rep_name = rep_exp_names[2]
curr_df = rep_dfs[curr_rep_name]
kp_detectors = curr_df.columns.to_list()
datasets = curr_df.index.to_list()
repeatiblity_data = {}
down_sample_list = list(curr_df[kp_detectors[0]].loc[datasets[0]].keys())
curr_df.head()

,Harris,ISS,SIFT,USIP
kitti,"{'0': {'repeatability': 0.9943601466, 'matched...","{'0': {'repeatability': 0.8043786583, 'matched...","{'0': {'repeatability': 0.4979568846, 'matched...","{'0': {'repeatability': 0.53125, 'matched': 27..."
oxford,"{'0': {'repeatability': 0.9967213115, 'matched...","{'0': {'repeatability': 0.9177577585, 'matched...","{'0': {'repeatability': 0.3496873643, 'matched...","{'0': {'repeatability': 0.53125, 'matched': 27..."
luco,"{'0': {'repeatability': 0.9912790698, 'matched...","{'0': {'repeatability': 0.8260521633, 'matched...","{'0': {'repeatability': 0.4786729858, 'matched...","{'0': {'repeatability': 0.246875, 'matched': 1..."


In [209]:
rep_dfs[rep_exp_names[1]]

,Harris,ISS,SIFT,USIP
kitti,"{'0': {'repeatability': 1.0, 'matched': 215.0,...","{'0': {'repeatability': 0.9982676407, 'matched...","{'0': {'repeatability': 0.8030529413, 'matched...","{'0': {'repeatability': 0.51796875, 'matched':..."
oxford,"{'0': {'repeatability': 1.0, 'matched': 120.4,...","{'0': {'repeatability': 0.9990068326, 'matched...","{'0': {'repeatability': 0.7582347066, 'matched...","{'0': {'repeatability': 0.532421875, 'matched'..."
luco,"{'0': {'repeatability': 1.0, 'matched': 344.0,...","{'0': {'repeatability': 0.9991327328, 'matched...","{'0': {'repeatability': 0.7772511848, 'matched...","{'0': {'repeatability': 0.237890625, 'matched'..."


In [210]:

curr_rep_name = rep_exp_names[0]
print(curr_rep_name)
curr_df = rep_dfs[curr_rep_name]
kp_detectors = curr_df.columns.to_list()
datasets = curr_df.index.to_list()
repeatiblity_data = {}
down_sample_list = list(curr_df[kp_detectors[0]].loc[datasets[0]].keys())
curr_df.head()



Transformation_invariant_Repeatablity_rotation_only


,Harris,ISS,SIFT,USIP
kitti,"{'0': {'repeatability': 1.0, 'matched': 213.2,...","{'0': {'repeatability': 0.9982762689, 'matched...","{'0': {'repeatability': 0.7870557963, 'matched...","{'0': {'repeatability': 0.50859375, 'matched':..."
oxford,"{'0': {'repeatability': 1.0, 'matched': 119.2,...","{'0': {'repeatability': 1.0, 'matched': 416.6,...","{'0': {'repeatability': 0.7609757023, 'matched...","{'0': {'repeatability': 0.534375, 'matched': 2..."
luco,"{'0': {'repeatability': 1.0, 'matched': 344.0,...","{'0': {'repeatability': 0.9989993326, 'matched...","{'0': {'repeatability': 0.7772511848, 'matched...","{'0': {'repeatability': 0.231640625, 'matched'..."


In [211]:

for curr_dataset in datasets:
    print("Dataset: ",curr_dataset)
    for curr_detector in kp_detectors:    
        curr_row = curr_df[curr_detector].loc[curr_dataset]
        print(curr_detector)
        for downsample in curr_row:
            #print(sigma)
            #print(curr_row[sigma]['repeatability'])
            
            if curr_dataset not in repeatiblity_data:
                repeatiblity_data[curr_dataset] ={}
            if curr_detector not in repeatiblity_data[curr_dataset]:
                repeatiblity_data[curr_dataset][curr_detector] = []
            repeatiblity_data[curr_dataset][curr_detector].append(curr_row[downsample]['repeatability'])

Dataset:  kitti
Harris
ISS
SIFT
USIP
Dataset:  oxford
Harris
ISS
SIFT
USIP
Dataset:  luco
Harris
ISS
SIFT
USIP


In [212]:
for curr_dataset in datasets:
    
    repeatiblity_curr_df = pd.DataFrame.from_dict(repeatiblity_data[curr_dataset])
    plt.figure()
    plt.plot(repeatiblity_curr_df)
    plt.xlabel("Rotation (in degree)")
    plt.ylabel("Repeatabilty %")
    x = np.arange(len(down_sample_list))
    plt.xticks(x, down_sample_list, rotation=-15)
    plt.legend(list(repeatiblity_curr_df.keys()))
    
    plt.title(curr_rep_name.replace("_", " ")+ " for "+curr_dataset)

    plt.savefig(extracted_results_dir+curr_rep_name+"_"+curr_dataset+"_plot.png")

/home/pranayspeed/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


In [213]:

curr_rep_name = rep_exp_names[1]
print(curr_rep_name)
curr_df = rep_dfs[curr_rep_name]
kp_detectors = curr_df.columns.to_list()
datasets = curr_df.index.to_list()
repeatiblity_data = {}
down_sample_list = list(curr_df[kp_detectors[0]].loc[datasets[0]].keys())


for curr_dataset in datasets:
    print("Dataset: ",curr_dataset)
    for curr_detector in kp_detectors:    
        curr_row = curr_df[curr_detector].loc[curr_dataset]
        print(curr_detector)
        for downsample in curr_row:
            #print(sigma)
            #print(curr_row[sigma]['repeatability'])
            
            if curr_dataset not in repeatiblity_data:
                repeatiblity_data[curr_dataset] ={}
            if curr_detector not in repeatiblity_data[curr_dataset]:
                repeatiblity_data[curr_dataset][curr_detector] = []
            repeatiblity_data[curr_dataset][curr_detector].append(curr_row[downsample]['repeatability'])

for curr_dataset in datasets:
    
    repeatiblity_curr_df = pd.DataFrame.from_dict(repeatiblity_data[curr_dataset])
    plt.figure()
    plt.plot(repeatiblity_curr_df)
    plt.xlabel("Translation (in meter)")
    plt.ylabel("Repeatabilty %")
    x = np.arange(len(down_sample_list))
    plt.xticks(x, down_sample_list, rotation=-15)
    plt.legend(list(repeatiblity_curr_df.keys()))
    
    plt.title(curr_rep_name.replace("_", " ")+ " for "+curr_dataset)

    plt.savefig(extracted_results_dir+curr_rep_name+"_"+curr_dataset+"_plot.png")


Transformation_invariant_Repeatablity_Translation_only
Dataset:  kitti
Harris
ISS
SIFT
USIP
Dataset:  oxford
Harris
ISS
SIFT
USIP
Dataset:  luco
Harris
ISS
SIFT
USIP


In [214]:
def generate_graphs_for_repeatability(curr_df, extracted_results_dir):
    kp_detectors = curr_df.columns.to_list()
    datasets = curr_df.index.to_list()
    repeatiblity_data = {}
    down_sample_list = list(curr_df[kp_detectors[0]].loc[datasets[0]].keys())


    for curr_dataset in datasets:
        print("Dataset: ",curr_dataset)
        for curr_detector in kp_detectors:    
            curr_row = curr_df[curr_detector].loc[curr_dataset]
            print(curr_detector)
            for downsample in curr_row:
                #print(sigma)
                #print(curr_row[sigma]['repeatability'])
                
                if curr_dataset not in repeatiblity_data:
                    repeatiblity_data[curr_dataset] ={}
                if curr_detector not in repeatiblity_data[curr_dataset]:
                    repeatiblity_data[curr_dataset][curr_detector] = []
                repeatiblity_data[curr_dataset][curr_detector].append(curr_row[downsample]['repeatability'])

    for curr_dataset in datasets:
        
        repeatiblity_curr_df = pd.DataFrame.from_dict(repeatiblity_data[curr_dataset])
        plt.figure()
        plt.plot(repeatiblity_curr_df)
        plt.xlabel("Rotation (in degree)")
        plt.ylabel("Repeatabilty %")
        x = np.arange(len(down_sample_list))
        plt.xticks(x, down_sample_list, rotation=-15)
        plt.legend(list(repeatiblity_curr_df.keys()))
        
        plt.title(curr_rep_name.replace("_", " ")+ " for "+curr_dataset)

        plt.savefig(extracted_results_dir+curr_rep_name+"_"+curr_dataset+"_plot.png")


In [215]:

curr_rep_name = rep_exp_names[2]
print(curr_rep_name)
curr_df = rep_dfs[curr_rep_name]
generate_graphs_for_repeatability(curr_df, extracted_results_dir)

Transformation_invariant_Repeatablity
Dataset:  kitti
Harris
ISS
SIFT
USIP
Dataset:  oxford
Harris
ISS
SIFT
USIP
Dataset:  luco
Harris
ISS
SIFT
USIP


### Resolution invariant plots

In [48]:

for curr_dataset in datasets:
    print("Dataset: ",curr_dataset)
    for curr_detector in kp_detectors:    
        curr_row = curr_df[curr_detector].loc[curr_dataset]
        print(curr_detector)
        for downsample in curr_row:
            #print(sigma)
            #print(curr_row[sigma]['repeatability'])
            
            if curr_dataset not in repeatiblity_data:
                repeatiblity_data[curr_dataset] ={}
            if curr_detector not in repeatiblity_data[curr_dataset]:
                repeatiblity_data[curr_dataset][curr_detector] = []
            repeatiblity_data[curr_dataset][curr_detector].append(curr_row[downsample]['repeatability'])


Dataset:  kitti
Harris
ISS
SIFT
USIP
Dataset:  oxford
Harris
ISS
SIFT
USIP
Dataset:  luco
Harris
ISS
SIFT
USIP


In [49]:
for curr_dataset in datasets:
    
    repeatiblity_curr_df = pd.DataFrame.from_dict(repeatiblity_data[curr_dataset])
    plt.figure()
    plt.plot(repeatiblity_curr_df)
    plt.xlabel("Standard Deviation")
    plt.ylabel("Repeatabilty %")
    x = np.arange(len(down_sample_list))
    plt.xticks(x, down_sample_list, rotation=-15)
    plt.legend(list(repeatiblity_curr_df.keys()))
    
    plt.title(curr_rep_name.replace("_", " ")+ " for "+curr_dataset)

    plt.savefig(extracted_results_dir+curr_rep_name+"_"+curr_dataset+"_plot.png")

In [42]:
rep_dfs[rep_exp_names[2]]

,Harris,ISS,SIFT,USIP
kitti,"{'repeatability': 0.9809161548, 'matched': 214...","{'repeatability': 0.7879849124, 'matched': 542...","{'repeatability': 0.234321224, 'matched': 49.4...","{'repeatability': 0.0455078125, 'matched': 23...."
oxford,"{'repeatability': 0.9945376352, 'matched': 119...","{'repeatability': 0.9129044213, 'matched': 381...","{'repeatability': 0.1689119924, 'matched': 15....","{'repeatability': 0.0538085938, 'matched': 27...."
luco,"{'repeatability': 0.9825581395, 'matched': 338...","{'repeatability': 0.808412665, 'matched': 2421...","{'repeatability': 0.2322274882, 'matched': 98....","{'repeatability': 0.016015625, 'matched': 8.2,..."


In [56]:

curr_rep_name = rep_exp_names[2]
curr_df = rep_dfs[curr_rep_name]
kp_detectors = curr_df.columns.to_list()
datasets = curr_df.index.to_list()
repeatiblity_data = {}
down_sample_list = list(curr_df[kp_detectors[0]].loc[datasets[0]].keys())
curr_df.head()

,Harris,ISS,SIFT,USIP
kitti,"{'repeatability': 0.9809161548, 'matched': 214...","{'repeatability': 0.7879849124, 'matched': 542...","{'repeatability': 0.234321224, 'matched': 49.4...","{'repeatability': 0.0455078125, 'matched': 23...."
oxford,"{'repeatability': 0.9945376352, 'matched': 119...","{'repeatability': 0.9129044213, 'matched': 381...","{'repeatability': 0.1689119924, 'matched': 15....","{'repeatability': 0.0538085938, 'matched': 27...."
luco,"{'repeatability': 0.9825581395, 'matched': 338...","{'repeatability': 0.808412665, 'matched': 2421...","{'repeatability': 0.2322274882, 'matched': 98....","{'repeatability': 0.016015625, 'matched': 8.2,..."


In [57]:

for curr_dataset in datasets:
    print("Dataset: ",curr_dataset)
    for curr_detector in kp_detectors:    
        curr_row = curr_df[curr_detector].loc[curr_dataset]["repeatability"]
        print(curr_detector)
        #print(curr_row)            
        if curr_dataset not in repeatiblity_data:
            repeatiblity_data[curr_dataset] ={}

        repeatiblity_data[curr_dataset][curr_detector]= curr_row


Dataset:  kitti
Harris
ISS
SIFT
USIP
Dataset:  oxford
Harris
ISS
SIFT
USIP
Dataset:  luco
Harris
ISS
SIFT
USIP


In [58]:
repeatiblity_data

{'kitti': {'Harris': 0.9809161548,
  'ISS': 0.7879849124,
  'SIFT': 0.234321224,
  'USIP': 0.0455078125},
 'oxford': {'Harris': 0.9945376352,
  'ISS': 0.9129044213,
  'SIFT': 0.1689119924,
  'USIP': 0.0538085938},
 'luco': {'Harris': 0.9825581395,
  'ISS': 0.808412665,
  'SIFT': 0.2322274882,
  'USIP': 0.016015625}}

In [ ]:
for curr_dataset in datasets:
    
    repeatiblity_curr_df = pd.DataFrame.from_dict(repeatiblity_data[curr_dataset])
    plt.figure()
    plt.plot(repeatiblity_curr_df)
    plt.xlabel("Standard Deviation")
    plt.ylabel("Repeatabilty %")
    x = np.arange(len(down_sample_list))
    plt.xticks(x, down_sample_list, rotation=-15)
    plt.legend(list(repeatiblity_curr_df.keys()))
    
    plt.title(curr_rep_name.replace("_", " ")+ " for "+curr_dataset)

    plt.savefig(extracted_results_dir+curr_rep_name+"_"+curr_dataset+"_plot.png")

In [59]:
repeatiblity_curr_df = pd.DataFrame.from_dict(repeatiblity_data)

In [62]:
repeatiblity_curr_df.to_json(extracted_results_dir+curr_rep_name+"_repeatablity_val.json")
repeatiblity_curr_df.to_latex(extracted_results_dir+curr_rep_name+"_repeatablity_val.latex")

In [66]:
# repeatiblity_curr_df.T.plot()
# plt.show()